# Parkinson's disease based on voice recording

## Data Description and Context:
Parkinson’s Disease (PD) is a degenerative neurological disorder marked by decreased dopamine levels in the brain. It manifests itself through a deterioration of movement, including the presence of tremors and stiffness. There is commonly a marked effect on speech, including dysarthria (difficulty articulating sounds), hypophonia (lowered volume), and monotone (reduced pitch range). Additionally, cognitive impairments and changes in mood can occur, and risk of dementia is increased.
Traditional diagnosis of Parkinson’s Disease involves a clinician taking a neurological history of the patient and observing motor skills in various situations. Since there is no definitive laboratory test to diagnose PD, diagnosis is often difficult, particularly in the early stages when motor effects are not yet severe. Monitoring progression of the disease over time requires repeated clinic visits by the patient. An effective screening process, particularly one that doesn’t require a clinic visit, would be beneficial. Since PD patients exhibit characteristic vocal features, voice recordings are a useful and non-invasive tool for diagnosis. If machine learning algorithms could be applied to a voice recording dataset to accurately diagnosis PD, this would be an effective screening step prior to an appointment with a clinician

## Domain:
Medicine
    
## Attribute Information:
* name - ASCII subject name and recording number
* MDVP:Fo(Hz) - Average vocal fundamental frequency
* MDVP:Fhi(Hz) - Maximum vocal fundamental frequency
* MDVP:Flo(Hz) - Minimum vocal fundamental frequency
* MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP - Several
* measures of variation in fundamental frequency
* MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA - Several measures of variation in amplitude
* NHR,HNR - Two measures of ratio of noise to tonal components in the voice
* status - Health status of the subject (one) - Parkinson's, (zero) - healthy
* RPDE,D2 - Two nonlinear dynamical complexity measures
* DFA - Signal fractal scaling exponent
* spread1,spread2,PPE - Three nonlinear measures of fundamental frequency variation 9. 
* car name: string (unique for each instance)

    
## Learning Outcomes:
● Exploratory Data Analysis

● Supervised Learning

● Ensemble Learning

## Objective:
Goal is to classify the patients into the respective labels using the attributes from their voice recordings

# Importing libraries

In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import scipy.stats as stats
import numpy as np
from pandas.api.types import is_numeric_dtype
from statsmodels.stats.proportion import proportions_ztest
import seaborn as sns

from matplotlib import pyplot as plt
from scipy.stats import gamma
from sklearn.preprocessing import StandardScaler
sns.set(color_codes=True)
%matplotlib inline

ModuleNotFoundError: No module named 'statsmodels'

# Loading data into dataframe

In [ ]:
df = pd.read_csv("../input/parkinsons/datasets_410614_786211_parkinsons.csv", sep=',')

In [ ]:
df.shape

#### We have a total of 23 features and 195 rows of data

In [ ]:
df.head()

## Some preprocessing

In [ ]:
df.info()

#### Observation: We can see that all the features except our target feature status is continous but we should convert status to object data type since it is a binary categorical feature 

## Checking for null values

In [ ]:
df.isnull().values.any()

In [ ]:
df.isnull().any()

In [ ]:
df.isnull().sum()

#### Observations: From above we can observe that there are no null values

## Check for duplicate names

In [ ]:
len(df['name'].unique())

In [ ]:
df.shape[0]

#### Observations: Since number of rows is equivalent to count of unique values of column 'name' , hence there are no duplicate values of feature 'name'

## Converting status to object data type

In [ ]:
df['status'].unique()

It is a binary categorical data as we observed with only 2 unique values i.e 0 and 1

In [ ]:
df['status'] = df['status'].astype('object')

In [ ]:
df.info(verbose = True)

# Exploratory Data Analysis(EDA)

## Functions to ease up plotting of various continous variables

In [ ]:
def Distribution_Continous_Variables(series,color,title):
    plt.figure(figsize=(10, 5))
    sns.distplot(series, color = color).set_title(title)
    
def Print_Summary(series,title,var):
    print(title)
    print('Count = {1}'.format(var,len(series)))
    print('Mean of {0} = {1}'.format(var,series.mean()))
    print('Median of {0} = {1}'.format(var,series.median()))
    print('Mode of {0} = {1}'.format(var,series.mode().values[0]))
    print('Skewness of {0} = {1}'.format(var, series.skew()))
    print('Excess Kurtosis of {0} = {1}'.format(var,series.kurtosis()))
    print(100*"*")

def Coeff_Variation(series,title,var):
    print('CV of {0} for {1} = {2}'.format(var,title,(series.std()/series.mean())*100))

def BoxPlot(**kwargs):
    plt.figure(figsize=(10, 5))
    sns.boxplot(x = kwargs['x'], \
                y = kwargs['y'], \
                data = kwargs['data'], \
                color = kwargs['color'], \
                hue = kwargs['hue']).set_title(kwargs['title'])    

def ViolinPlot(**kwargs):
    plt.figure(figsize=(10, 5))
    sns.violinplot(x = kwargs['x'], \
                y = kwargs['y'], \
                data = kwargs['data'], \
                color = kwargs['color'], \
                hue = kwargs['hue']).set_title(kwargs['title']) 
    
        
def CountPlot(**kwargs):
    plt.figure(figsize=(10, 5))
    sns.countplot(y=kwargs['y'], \
                    hue=kwargs['hue'], \
                    data=kwargs['data']).set_title(kwargs['title'])
    
   

In [ ]:
list_of_non_object_cols = df.loc[:, df.dtypes != 'object'].columns.tolist()

## Univariate Analysis

### Plots to analyse impact of continous on the status

In [ ]:
for col in list_of_non_object_cols:
    Distribution_Continous_Variables(df[df['status']==0][col],"green","Distribution of {} in the dataset for healthy patients"\
                                    .format(col))
    Distribution_Continous_Variables(df[df['status']==1][col],"red","Distribution of {} in the dataset for patients with parkinson's disease"\
                                    .format(col))

#### Observations: 
* From the above set of graphs we can observe that majority of our continous variables are skewed to the right and we cannot observe any perfectly normally distributed continous variable.
* This somehow provides us a rough idea that our dataset is full of outliers that we need to get rid of.

#### Next we will try to get a numerical summary pertaining to our features to further consolidate our observation

In [ ]:
df.describe().T

In [ ]:
for col in list_of_non_object_cols:
    Print_Summary(df[df['status']==0][col],"Numerical Summary of {} for healthy patients"\
                                    .format(col),col)
    Print_Summary(df[df['status']==1][col],"Numerical Summary of {} for patients with parkinson's disease"\
                                    .format(col),col)

In [ ]:
CountPlot(y = 'status',\
          hue = 'status',\
          data = df,\
          title = "Count comparison of healthy people vs people with parkinson's disease")

In [ ]:
print("Percentage of patients diagnosed as healthy = {0:.2f}%".format((df[df.status == 0].shape[0]/df.shape[0])*100))

In [ ]:
print("Percentage of patients diagnosed with Parkinson's disease = {0:.2f} %".format((df[df.status == 1].shape[0]/df.shape[0])*100))

#### Observations:
* The data is imbalanced since we have 24.6 % of healthy patients while 75.38 % are sick.
* PPE for healthy patients,spread2 for both types of subjects, RPDE for both types of subjects tend to have a normal distribution as per their statisticals values while other features seem to be skewed.

* #### We will plot boxplot now to observe if our continous features have outliers or not. The boxplot uses IQR method to   detect outliers. Also boxplot would give a visual representation of the five point summary
* #### We will also plot violinplot along with boxplots to compare the distributions as well.

In [ ]:
for col in list_of_non_object_cols:
    BoxPlot(x = 'status',\
            y = col,\
            data = df,\
            color = 'orange',\
            hue = 'status',\
            title = 'Boxplot of {}'.format(col))

    ViolinPlot(x = 'status',\
            y = col,\
            data = df,\
            color = 'orange',\
            hue = 'status',\
            title = 'Violinplot of {}'.format(col))

#### Observations : Except MDVP:Fo(Hz),MDVP:Flo(Hz),RPDE and DFA every other feature has outliers as per IQR method.

### Coefficient of Variation(CV) of continous variables
* CV is relative comparison of the distributions with respect to their standard deviations.
* This is unit agnostic i.e units do not have any impact on its value
* Greater the number ,greater is the variability

In [ ]:
for col in list_of_non_object_cols:
    Coeff_Variation(df[df['status']==0][col],"for healthy subjects".format(col),col)
    Coeff_Variation(df[df['status']==1][col],"for patients with Parkinson's disease".format(col),col)
    print(100*"*")

## BiVariate Analysis

### Pairplot with Status as hue

In [ ]:
sns.pairplot(df, hue = 'status')
plt.show()

#### Observations: From the plot we could see that there might be positive or negative correlation between certain variables.Like between spread1 and PPE there seems to be a positive correlation. We will further analyse the correlation by meausring pearson coefficient of correlation between all features

### Measuring correlation

In [ ]:
plt.figure(figsize=(20, 20))
df_corr = df.corr(method='pearson')
ax = sns.heatmap(df_corr, annot=True, cmap='YlGnBu')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

#### Let's plot highly positively correlated or negatively correlated features. We will use 0.80 and -0.80 as our threshold correlation coefficient for positive and negative correlations respectively. Usually ensemble mehtods handle correlations well but highly correlated features may create problems.

In [ ]:
plt.figure(figsize=(12, 10))
df_corr = df.corr(method='pearson')
ax = sns.heatmap(df_corr[(df_corr >= 0.80) | (df_corr <= -0.80)], annot=True, cmap='magma')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

#### Observations: We can clearly observe here that there are plenty of highly correlated features in our dataset which we need to remove before as part of the data cleaning before we can utilise our dataset for machine learning. High correlation also happens due to imbalanced dataset. So we will try to balance it by using oversampling and then would observe our correlation heatmap again

### Preprocessing and Feature engineering before Model Building

#### Why we do not need to treat the imbalanced dataset via oversampling?

* We observed in the countplot in the EDA section that there is an imbalance between the count of patients and healthy subjects
* To counter this we could have used SMOTE oversampling method which produces sythetic variables for minority class by using k nearest neighbors. 
* But a dataset like that would make our model less efficient while treating real life datasets where we may only find imbalance of datapoints between patients and healthy subjects.

#### We will remove column name as it is not anyway useful for us in model building

In [ ]:
df.drop(columns=['name'],inplace=True)

#### Removing outliers using z-score method:We detected outliers in many features by using IQR method. Since we have less number of rows in our dataset , we ould rather use z-score method to detect outliers. If the z score value is 3 and above i.e if a value is beyond 3 standard deviations we would drop those rows.

In [ ]:
non_obj_cols = df.loc[:, df.dtypes != 'object'].columns.tolist()
z = np.abs(stats.zscore(df[non_obj_cols]))

In [ ]:
df_clean = df[(z<3).all(axis=1)]
df_clean.shape,df.shape

By using z score method we have removed around 14 outlier rows

#### After removing outliers let us observe the correlation matrix again

In [ ]:
plt.figure(figsize=(12, 10))
df_corr = df_clean.corr(method='pearson')
ax = sns.heatmap(df_corr[(df_corr >= 0.80) | (df_corr <= -0.80)], annot=True, cmap='magma')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

#### Observations: The correlation matrix after removing outliers contains fewer highly correlated features.This proves the fact that correlation coefficients do get affected by the presence of outliers.

### Let's remove highly correlated features (both negative and positive) from the cleaned up data

In [ ]:
upper_triangle = df_corr.where(np.triu(np.ones(df_corr.shape),k=1).astype('bool'))
cols_to_drop = [column for column in upper_triangle.columns \
                if any((upper_triangle[column] >= 0.80) | (upper_triangle[column] <= -0.80))]

In [ ]:
cols_to_drop

In [ ]:
df_copy = df_clean.drop(columns=cols_to_drop).copy()

### Let's plot the correlation matrix again after removing correlated variables

In [ ]:
plt.figure(figsize=(12, 10))
df_corr = df_copy.corr(method='pearson')
ax = sns.heatmap(df_corr, annot=True, cmap='YlGnBu')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

### Shape after removing correlated features

In [ ]:
df_copy.shape

Now after cleaning we are left with 13 features and 181 rows

### Standardizing continous variables

In [ ]:
X = df_copy.copy()
cont_feat = X.loc[:,X.dtypes != 'object'].columns.tolist()
features = X[cont_feat]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

In [ ]:
X[cont_feat] = features

In [ ]:
X.head()

In [ ]:
CountPlot(y = 'status',\
          hue = 'status',\
          data = X,\
          title = "Count comparison of healthy people vs people with parkinson's disease")

In [ ]:
print("Percentage of patients diagnosed as healthy = {0:.2f}%".format((X[X.status == 0].shape[0]/X.shape[0])*100))

In [ ]:
print("Percentage of patients diagnosed with Parkinson's disease = {0:.2f} %".format((X[X.status == 1].shape[0]/X.shape[0])*100))

#### Observations : After performing some data cleaning and feature engineering we can see that percentage distribution between 2 status has slightly improved

# Building and Analysis of Machine learning models

## Train-Test split (70:30)

In [ ]:
y = X['status'] #------Target variable-------------#
X = X.drop(columns=['status']) #------Features--------#

In [ ]:
y = y.astype('int')

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42, stratify=y)

#### Analysing the count after split for both test and train data

In [ ]:
print("Percentage of patients diagnosed as healthy in train dataset = {0:.2f}%".format((y_train[y_train == 0].shape[-1]/y_train.shape[-1])*100))

In [ ]:
print("Percentage of patients diagnosed with Parkinson's disease in train dataset = {0:.2f} %".format((y_train[y_train == 1].shape[-1]/y_train.shape[-1])*100))

In [ ]:
print("Percentage of patients diagnosed as healthy in test dataset = {0:.2f}%".format((y_test[y_test == 0].shape[-1]/y_test.shape[-1])*100))

In [ ]:
print("Percentage of patients diagnosed with Parkinson's disease in test dataset = {0:.2f} %".format((y_test[y_test == 1].shape[-1]/y_test.shape[-1])*100))

#### Observation :  The split in both test and train data is almost in same proportion

## Logistic regression model

## Conclusion:
* The f1-score is a better metric to measure the performance of our model when compared with accuracy, because our data is imbalanced<li>In case if a person is sick and if our model predicts him to be healthy then that would be a risky model
* Here in this case the f1 score for the patients with parkinson's is better at 0.91 i.e the patients who actually have parkinson's have higher chance of getting diagnosed correctly by our logistic regression model.
* Also we have a decent macro average f1 score at 0.79 which signifies that our model can diagnose a person and predict whether he is healthy or sick with high accuracy.
* The weighted average f1 takes into account the class imbalance and assigns higher weightage to the minority class(in this case class of sick person). The score of weighted average of 85% further consolidates our point that this model performs well while classifying the sick.
* For the sake of convinience we ould consider f1-scores as the metric for the model performance moving ahead.<li>We would be more interested in f1-scores of status 1(sick subjects) and macro avg f1 scores

## K nearest nighbour model

Test accuracy is at 96% but it is not a correct metric to measure the performance of our model

## Confusion Matrix

### Conclusion : Our model's macro f1 score and the f1 score in case of status 1(sick subjects) is 0.96 and 0.97 respectively which is very good when compared with logistic regression model

## Support Vector Machine model

We will use gridsearchcv for hyperparameter tuning since we have 2 hyperparameters i.e C and gamma

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['sigmoid']}  
  
svm = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
svm.fit(X_train, y_train) 

In [ ]:
print(svm.best_params_)

In [ ]:
print(svm.best_estimator_)

In [ ]:
svm = SVC(kernel='rbf', C=10, gamma=0.1)
svm.fit(X_train,y_train)
print(svm.score(X_test, y_test))

## Confusion Matrix

In [ ]:
plt.figure(figsize=(10, 5))
y_pred = svm.predict(X_test)
cnf_matrix = confusion_matrix(y_test, y_pred)
p = sns.heatmap(cnf_matrix, annot=True, cmap="YlGnBu" ,fmt='g')
bottom, top = ax.get_ylim()
p.set_ylim(2, 0.0)
plt.title('Confusion matrix')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
import joblib

model_filename = 'parkinson_svm.pkl'
joblib.dump(svm, model_filename)

### Conclusion : Our model's macro f1 score and the f1 score in case of status 1(sick subjects) is 0.99 and 0.98 respectively which is very good when compared with both logistic regression model and the k nearest neighbor model

# Training a meta classifier

Now we will be using a meta-classifier that would take different models as estimators and would stack them up to come up with the best classifier model. For that we would be using Stacking classifier

* Above we have stacked logistic regression model and k nearest neighbr model and as a result of which our accuracy is 98%
* But we need to check f1 score in order to confirm if our model is better than svm model or not

### Observation:
* So after stacking the logistic regression and knn model(who were not as good as svm model), we got f1 scores almost similar to that of the SVM model
* This proves that ensembling 2 inferior models could result in a superior model

## Using boosting ensemble method

# Final Conclusion and model comparison

* The f1-score is a better metric to measure the performance of our model when compared with accuracy, because our data is imbalanced
* In case if a person is sick and if our model predicts him to be healthy then that would be a risky model, hence we must consider F1 score for sick subjects as well as Macro average F1 score to rate a particular model.
* From above table we could observe that almost all of our models perform decently on the given dataset
* But SVM and our meta classifer where we stacked logistic regression model and k nearest neigbor model perform better than the other models.
* Hence, we can either pick SVM model or the simple stacked ensemble model to predict whether a subject is suffering from Parkinson's disease or not